In [ ]:
import psycopg2
import csv

In [ ]:
db = psycopg2.connect(
    user="lvs215",
    password="",
    host="localhost",
    port="5432",
    database="lvs215"
)

In [ ]:
queries = [
    {
        "name": "reproducibility",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%') 
                """,
        "run": False
    },
    {
        "name": "reproducibility-system",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%') 
                AND
                (lower(ps.title) LIKE '%system%' OR lower(ps.abstract) LIKE '%system%') 
                """,
        "run": False
    },
    {
        "name": "reproducibility-systems",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%') 
                AND
                (lower(ps.title) LIKE '%systems%' OR lower(ps.abstract) LIKE '%systems%') 
                """,
        "run": False
    },
    {
        "name": "reproducibility-or-repeatability",
        "query": """
                SELECT ps.id, ps.title, ps.abstract, ps.year, ps.venue, ps.doi, ps.n_citations
                FROM publications ps
                WHERE 
                ( 
                    (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%')
                    OR
                    (lower(ps.title) LIKE '%repeatability%' OR lower(ps.abstract) LIKE '%repeatability%')
                )
                """,
        "run": True
    },
    {
        "name": "reproducibility-or-repeatability-or-replicability",
        "query": """
                SELECT ps.id, ps.title, ps.abstract, ps.year, ps.venue, ps.doi, ps.n_citations
                FROM publications ps
                WHERE 
                ( 
                    (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%')
                    OR
                    (lower(ps.title) LIKE '%repeatability%' OR lower(ps.abstract) LIKE '%repeatability%')
                    OR
                    (lower(ps.title) LIKE '%replicability%' OR lower(ps.abstract) LIKE '%replicability%')
                )
                """,
        "run": True
    },
    {
        "name": "reproducibility-or-repeatability-system",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                ( 
                    (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%')
                    OR
                    (lower(ps.title) LIKE '%repeatability%' OR lower(ps.abstract) LIKE '%repeatability%')
                )
                AND
                (lower(ps.title) LIKE '%system%' OR lower(ps.abstract) LIKE '%system%') 
                """,
        "run": False
    },
    {
        "name": "reproducibility-or-repeatability-systems",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                ( 
                    (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%')
                    OR
                    (lower(ps.title) LIKE '%repeatability%' OR lower(ps.abstract) LIKE '%repeatability%')
                )
                AND
                (lower(ps.title) LIKE '%systems%' OR lower(ps.abstract) LIKE '%systems%') 
                """,
        "run": False
    }
]

In [ ]:
def test_query(query_dict):
    print(query_dict['name'])
    query_results = []
    if not query_dict['run']:
        print('skipped')
        return
    with db.cursor() as cursor:
        cursor.execute(query_dict["query"])
        field_names = [field[0] for field in cursor.description]
        query_results = cursor.fetchall()
    print("General Statistics:", "\n", "result length", len(query_results), "\n")
    with open(f"{query_dict['name']}.csv", 'w+', newline ='') as file:
        write = csv.writer(file)
        write.writerow(field_names)
        write.writerows(query_results)
    for pub in query_results[1:10]:
        print(pub)
    

In [5]:
for q in queries:
    print("#"*80, "\n")
    test_query(q)
    print("\n")

################################################################################ 

reproducibility
skipped


################################################################################ 

reproducibility-system
skipped


################################################################################ 

reproducibility-systems
skipped


################################################################################ 

reproducibility-or-repeatability
General Statistics: 
 result length 150 

('conf/cvpr/LeeC09', 'Histogram-based interest point detectors', 'We present a new method for detecting interest points using histogram information. Unlike existing interest point detectors, which measure pixel-wise differences in image intensity, our detectors incorporate histogram-based representations, and thus can find image regions that present a distinct distribution in the neighborhood. The proposed detectors are able to capture large-scale structures and distinctive textured patterns, an